In [1]:
!which jupyter

/home/ubuntu/thesis_env2/bin/jupyter


In [2]:
import pandas as pd
import numpy as np
import datetime
import spacy
import en_core_web_sm
#from spacy import displacy

In [3]:
df = pd.read_csv('./data/covid19_articles_20201231.csv')

# Date to datetime
df['date'] = pd.to_datetime(df['date'])

# Drop duplicates
num_articles_pre_drop = len(df)
df = df.drop_duplicates(subset='content')
print('no. articles dropped: ', num_articles_pre_drop - len(df))
print('no. of articles: ', len(df))

# Drop articles where text longer than 1,000,000 characters
df = df[df['content'].apply(lambda x: len(x) <= 1000000)].copy()
print('no. of articles after dropping long articles: ', len(df))

print(df.info())

no. articles dropped:  127
no. of articles:  368920
no. of articles after dropping long articles:  368918
<class 'pandas.core.frame.DataFrame'>
Int64Index: 368918 entries, 0 to 369046
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   author      181789 non-null  object        
 1   date        368918 non-null  datetime64[ns]
 2   domain      368918 non-null  object        
 3   title       368833 non-null  object        
 4   url         368918 non-null  object        
 5   content     368918 non-null  object        
 6   topic_area  368918 non-null  object        
dtypes: datetime64[ns](1), object(6)
memory usage: 22.5+ MB
None


In [4]:
df.index

Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,
            ...
            369037, 369038, 369039, 369040, 369041, 369042, 369043, 369044,
            369045, 369046],
           dtype='int64', length=368918)

In [5]:
# Get index numbers of articles to label
np.random.seed(777)
article_indices = np.random.choice(df.index, size=1000, replace=False)
article_indices

array([272801, 200452, 360615, 348240, 150912,   5919,  30166, 136359,
       208183,  23000, 330542,  58596, 286411, 261343, 113511,  42006,
       102479, 241927, 341037, 291682, 212144, 302846, 110625, 201788,
       170802, 159684, 333166, 220772, 237146, 129492, 313847, 156620,
        36890, 340034, 163144, 360941,  55550, 347657,  47979, 182837,
       134691, 160045, 286915, 278396, 233365, 286431,  83397, 166869,
       246232, 308226,  84557, 203388, 170380, 213355, 158274,  76833,
       156025, 264723, 117743,  94017,  47533,  19442, 356015, 167369,
       157731, 221842, 126826, 194484, 140473, 190886, 286245, 165741,
       243097, 225028, 116166, 202311,  50746, 323505,  24801, 306356,
       278877, 232675,  35740,  62652, 148690, 266123, 204738, 115954,
        23313, 190061, 141887, 187155, 171153, 169507, 354893, 160755,
       187739,  89199, 326662, 129575,  41302, 264705, 229588, 249778,
       245717, 174209, 152067, 205808,   7392, 190235, 107862, 276782,
      

In [8]:
# Load wip file
file_latest = 'df_lab_2021_07_11_21_20.csv'
df_lab = pd.read_csv('./data/' + file_latest, index_col=0)

In [9]:
df_lab

,author,date,domain,title,url,content,topic_area,org_names_listed,org_names_listed_uo
272801,NaN,2020-08-15,finance.yahoo,Julen Lopetegui: We must be at our best to bea...,https://finance.yahoo.com/news/julen-lopetegui...,Head coach Julen Lopetegui knows Sevilla will ...,business,[],[]
200452,NaN,2020-07-11,finance.yahoo,Gallagher Premiership teams will not be punish...,https://uk.finance.yahoo.com/news/gallagher-pr...,Gallagher Premiership clubs will have flexibil...,business,[],[]
360615,NaN,2020-12-14,marketscreener,"Don't stockpile food, minister tells British s...",https://www.marketscreener.com/quote/currency/...,"Britain and the EU agreed on Sunday to ""go the...",business,['Tesco PLC'],['Tesco']
348240,Joe Hoppe,2020-11-23,marketwatch,"Wynnstay Properties profit halves, lifts dividend",https://www.marketwatch.com/story/wynnstay-pro...,Wynnstay Properties PLC said Monday that its f...,business,['Wynnstay Properties Plc'],['Wynnstay Properties PLC']
150912,NaN,2020-06-11,finance.yahoo,U.K. Scientists Defy Johnson to Speak Out on V...,https://ca.finance.yahoo.com/news/johnson-unde...,(Bloomberg) -- U.K. Prime Minister Boris Johns...,business,[],[]
...,...,...,...,...,...,...,...,...,...
169245,By Reuters,2020-06-24,nytimes,"With No Tourists to Watch Migration, Kenyan Op...",https://www.nytimes.com/reuters/2020/06/24/bus...,NAIROBI — For Kenyan-based safari operator Saf...,business,NaN,NaN
306948,NaN,2020-09-12,marketscreener,"Pfizer, BioNTech propose expanding COVID-19 va...",https://www.marketscreener.com/news/latest/Pfi...,Pfizer Inc and BioNTech SE\non Saturday propos...,business,NaN,NaN
111840,NaN,2020-05-13,marketscreener,Ameresco : Announces Commercial Operations of ...,https://www.marketscreener.com/AMERESCO-INC-64...,Beale Hill Wind Farm is the first renewable ge...,business,NaN,NaN
72139,NaN,2020-04-16,marketscreener,"L'Oréal: News release: ""First Quarter 2020 Sales""",https://www.marketscreener.com/quote/stock/L-O...,"Clichy, 16 April 2020 at 6:00 p.m. First Quart...",business,NaN,NaN


In [10]:
ner_processor = en_core_web_sm.load()

In [ ]:
['AFP', 'Associated Press', 'BBC', 'Bloomberg', 'CNN', 'Daily Express', 'Ducker Frontier', 'Financial Times', 'Guardian', 'Marketwatch', 'NBC', 
 'Prudent Speculator', 'Reuters', 'RNZ', 'Sky News', 'SpreadEx', 'The Daily Mail', 'The Sun', 'The Canadian Press', 'LBC'] # Reuters also listed

['Aberdeen Standard Investments', 'Australian Bureau of Statistics', 'Commonwealth Financial Network', 'Federal Reserve', 'National Health Service', 
 'National Rail', 'New York Stock Exchange' 'NHS', 'Oxford University', 'Scientific Advisory Group for Emergencies', 'Securities and Exchange Commission', 
 'The Bank of England', 'The British Retail Consortium', 'The Office for Budget Responsibility', 'The Organisation for Economic Co-operation and Development', 
 'The Reserve Bank of Australia', 'The Vaccine Group', 'Toronto Stock Exchange', 'University of Plymouth', 'Eastspring Investments', 'NATO', 'FTSE', 'SEC', 
 'CDC', 'NYSE', 'C.D.C', 'W.H.O']


In [513]:
num = 299
article_index = article_indices[num]
print(article_index)
print(df_lab.loc[article_index]['org_names_listed'])

article = df_lab.loc[article_index]['content']
spacy.displacy.render(ner_processor(article), style='ent', options = {"ents": ["ORG"]})

311650
nan


In [514]:
#article_index = 311650
listed_orgs =   []
listed_orgs_uo =   []

df_lab.at[article_index,'org_names_listed'] = listed_orgs
df_lab.at[article_index,'org_names_listed_uo'] = listed_orgs_uo

In [515]:
df_lab.loc[article_indices[290:300]]

,author,date,domain,title,url,content,topic_area,org_names_listed,org_names_listed_uo
102110,NaN,2020-05-07,marketscreener,Fiverr International : Announces First Quarter...,https://www.marketscreener.com/FIVERR-INTERNAT...,"Fiverr International Ltd. (NYSE: FVRR), the co...",business,[Fiverr International Ltd.],[Fiverr International Ltd.]
95788,NaN,2020-05-04,marketscreener,Gilead Sciences : CEO Says Remdesivir Will Be ...,https://www.marketscreener.com/GILEAD-SCIENCES...,The Chief Executive Officer of Gilead Sciences...,business,"[Gilead Sciences, Inc.]",[Gilead Sciences]
232920,NaN,2020-07-24,marketscreener,"Industrials Down As Tensions Between The U.S.,...",https://www.marketscreener.com/news/Industrial...,Shares of industrial and transportation compan...,business,"[Honeywell International Inc., Airbus SE]","[Honeywell International, Airbus]"
191712,NaN,2020-07-08,finance.yahoo,Edited Transcript of ESSO.BK earnings conferen...,https://ca.finance.yahoo.com/news/edited-trans...,Q1 2020 Esso Thailand PCL Earnings Call Bangko...,business,[],[]
120373,NaN,2020-05-19,marketscreener,TEGNA : Announces Quarterly Dividend,https://www.marketscreener.com/TEGNA-INC-22752...,TEGNA Inc.'s (NYSE: TGNA) Board of Directors t...,business,[TEGNA Inc.],[TEGNA Inc.]
306195,NaN,2020-09-11,finance.yahoo,More Airfreight Providers Put Premium On Pharm...,https://finance.yahoo.com/news/more-airfreight...,"Several airlines, logistics companies and airp...",business,"[Air France-KLM SA, Nippon Express Co., Ltd., ...","[Air France-KLM, Nippon Express Co., DSV, LATA..."
258600,NaN,2020-08-04,finance.yahoo,Fleabag fans offered signed original show post...,https://uk.finance.yahoo.com/news/fleabag-fans...,Fleabag fans have the chance to get hold of th...,business,[],[]
285259,NaN,2020-08-24,finance.yahoo,"Yes, your houseplants can thrive. An expert sh...",https://finance.yahoo.com/news/yes-houseplants...,"On paper, it seems so easy: Give your housepla...",business,[],[]
348859,Opinion by Erin Bromage,2020-11-24,cnn,Opinion: You shouldn't gather this Thanksgivin...,https://www.cnn.com/2020/11/24/opinions/covid-...,Erin Bromage is an associate professor of biol...,general,[],[]
311650,Kevin Breuninger@KevinWilliamB,2020-09-15,cnbc,Trump: No regrets about Covid response: 'I thi...,https://www.cnbc.com/2020/09/15/trump-claims-h...,President Donald Trump on Tuesday denied that ...,finance,[],[]


In [516]:
#df_lab.loc[article_index]

# Articles not done - earnings call transcripts
[191712]

[191712]

In [517]:
date_label = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M")
file_name = 'df_lab_' + date_label + '.csv'
print(file_name)
df_lab.to_csv('./data/' + file_name)

df_lab_2021_07_12_18_43.csv


In [518]:
file_latest = 'df_lab_2021_07_12_18_43.csv' #299
df_check = pd.read_csv('./data/' + file_latest, index_col=0)
print(sum(df_check['org_names_listed'].isna()))
df_check

700


,author,date,domain,title,url,content,topic_area,org_names_listed,org_names_listed_uo
272801,NaN,2020-08-15,finance.yahoo,Julen Lopetegui: We must be at our best to bea...,https://finance.yahoo.com/news/julen-lopetegui...,Head coach Julen Lopetegui knows Sevilla will ...,business,[],[]
200452,NaN,2020-07-11,finance.yahoo,Gallagher Premiership teams will not be punish...,https://uk.finance.yahoo.com/news/gallagher-pr...,Gallagher Premiership clubs will have flexibil...,business,[],[]
360615,NaN,2020-12-14,marketscreener,"Don't stockpile food, minister tells British s...",https://www.marketscreener.com/quote/currency/...,"Britain and the EU agreed on Sunday to ""go the...",business,['Tesco PLC'],['Tesco']
348240,Joe Hoppe,2020-11-23,marketwatch,"Wynnstay Properties profit halves, lifts dividend",https://www.marketwatch.com/story/wynnstay-pro...,Wynnstay Properties PLC said Monday that its f...,business,['Wynnstay Properties Plc'],['Wynnstay Properties PLC']
150912,NaN,2020-06-11,finance.yahoo,U.K. Scientists Defy Johnson to Speak Out on V...,https://ca.finance.yahoo.com/news/johnson-unde...,(Bloomberg) -- U.K. Prime Minister Boris Johns...,business,[],[]
...,...,...,...,...,...,...,...,...,...
169245,By Reuters,2020-06-24,nytimes,"With No Tourists to Watch Migration, Kenyan Op...",https://www.nytimes.com/reuters/2020/06/24/bus...,NAIROBI — For Kenyan-based safari operator Saf...,business,NaN,NaN
306948,NaN,2020-09-12,marketscreener,"Pfizer, BioNTech propose expanding COVID-19 va...",https://www.marketscreener.com/news/latest/Pfi...,Pfizer Inc and BioNTech SE\non Saturday propos...,business,NaN,NaN
111840,NaN,2020-05-13,marketscreener,Ameresco : Announces Commercial Operations of ...,https://www.marketscreener.com/AMERESCO-INC-64...,Beale Hill Wind Farm is the first renewable ge...,business,NaN,NaN
72139,NaN,2020-04-16,marketscreener,"L'Oréal: News release: ""First Quarter 2020 Sales""",https://www.marketscreener.com/quote/stock/L-O...,"Clichy, 16 April 2020 at 6:00 p.m. First Quart...",business,NaN,NaN


In [523]:
df_lab_latest = df_lab.copy()
df_lab_latest.to_csv('./data/df_lab_latest.csv')

,author,date,domain,title,url,content,topic_area,org_names_listed,org_names_listed_uo
272801,NaN,2020-08-15,finance.yahoo,Julen Lopetegui: We must be at our best to bea...,https://finance.yahoo.com/news/julen-lopetegui...,Head coach Julen Lopetegui knows Sevilla will ...,business,[],[]
200452,NaN,2020-07-11,finance.yahoo,Gallagher Premiership teams will not be punish...,https://uk.finance.yahoo.com/news/gallagher-pr...,Gallagher Premiership clubs will have flexibil...,business,[],[]
360615,NaN,2020-12-14,marketscreener,"Don't stockpile food, minister tells British s...",https://www.marketscreener.com/quote/currency/...,"Britain and the EU agreed on Sunday to ""go the...",business,['Tesco PLC'],['Tesco']
348240,Joe Hoppe,2020-11-23,marketwatch,"Wynnstay Properties profit halves, lifts dividend",https://www.marketwatch.com/story/wynnstay-pro...,Wynnstay Properties PLC said Monday that its f...,business,['Wynnstay Properties Plc'],['Wynnstay Properties PLC']
150912,NaN,2020-06-11,finance.yahoo,U.K. Scientists Defy Johnson to Speak Out on V...,https://ca.finance.yahoo.com/news/johnson-unde...,(Bloomberg) -- U.K. Prime Minister Boris Johns...,business,[],[]
...,...,...,...,...,...,...,...,...,...
169245,By Reuters,2020-06-24,nytimes,"With No Tourists to Watch Migration, Kenyan Op...",https://www.nytimes.com/reuters/2020/06/24/bus...,NAIROBI — For Kenyan-based safari operator Saf...,business,NaN,NaN
306948,NaN,2020-09-12,marketscreener,"Pfizer, BioNTech propose expanding COVID-19 va...",https://www.marketscreener.com/news/latest/Pfi...,Pfizer Inc and BioNTech SE\non Saturday propos...,business,NaN,NaN
111840,NaN,2020-05-13,marketscreener,Ameresco : Announces Commercial Operations of ...,https://www.marketscreener.com/AMERESCO-INC-64...,Beale Hill Wind Farm is the first renewable ge...,business,NaN,NaN
72139,NaN,2020-04-16,marketscreener,"L'Oréal: News release: ""First Quarter 2020 Sales""",https://www.marketscreener.com/quote/stock/L-O...,"Clichy, 16 April 2020 at 6:00 p.m. First Quart...",business,NaN,NaN
